In [ ]:
%matplotlib notebook

import time

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# read dataset
def load_data():
    """
    TODO
    """
    pass

In [ ]:
def random_test_train_split(data):
    """
    Splits the data into test and training sets, assuming last column is the label
    
    Parameters:
    -----------
    data: pandas.DataFrame
        consolidated dataset returned by load_data() or normalize_data()
    """
    x_cols = data.columns[:-1]
    X = data[x_cols]
    
    y_cols = data.columns[-1:]
    y = data[y_cols]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test

In [ ]:
def normalize_data(data):
    """
    Centers continuous variables around 0, and scales by 1/std
    
    Parameters:
    -----------
    data: pandas.DataFrame
        consolidated dataset returned by load_data()

    TODO: handle categorical variables
    """
    t_data = data - np.mean(data)
    t_data = t_data / np.std(data)
    return t_data

In [ ]:
def forwardprop(X):
    """
    yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h1    = tf.contrib.layers.fully_connected(X, 128,
                activation_fn=tf.nn.relu,
                normalizer_fn=tf.contrib.layers.batch_norm,
                normalizer_params=None,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                weights_regularizer=tf.contrib.layers.l2_regularizer(0.0),
                biases_initializer=tf.zeros_initializer(),
                biases_regularizer=None,
                reuse=None,
                variables_collections=None,
                outputs_collections=None,
                trainable=True,
                scope=None
            )
    h2 = tf.contrib.layers.fully_connected(h1, 64,
                activation_fn=tf.relu,
                normalizer_fn=tf.contrib.layers.batch_norm,
                normalizer_params=None,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                weights_regularizer=tf.contrib.layers.l2_regularizer(0.0),
                biases_initializer=tf.zeros_initializer(),
                biases_regularizer=None,
                reuse=None,
                variables_collections=None,
                outputs_collections=None,
                trainable=True,
                scope=None
            )
    h3 = tf.contrib.layers.fully_connected(h2, 1,
                activation_fn=tf.nn.sigmoid,
                normalizer_fn=None,
                normalizer_params=None,
                weights_initializer=tf.contrib.layers.xavier_initializer(),
                weights_regularizer=tf.contrib.layers.l2_regularizer(0.0),
                biases_initializer=tf.zeros_initializer(),
                biases_regularizer=None,
                reuse=None,
                variables_collections=None,
                outputs_collections=None,
                trainable=True,
                scope=None
            )
    yhat = h3
    return yhat

In [ ]:
dataset = load_data()
t_data = normalize_data(dataset)
train_X, test_X, train_y, test_y = random_test_train_split(t_data)

In [ ]:
# Layer's sizes
x_size = train_X.shape[1]   # Number of input nodes
y_size = train_y.shape[1]   # Number of outcomes


In [ ]:
print(train_X.shape)

In [ ]:
# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

In [ ]:
# Forward propagation
yhat, h    = forwardprop(X)
# predict = tf.argmax(yhat, axis=1)

In [ ]:
# Backward propagation
cost    = tf.nn.softmax_cross_entropy_with_logits(y, yhat)
updates = tf.train.AdamOptimizer(0.01).minimize(cost)


In [ ]:
# Run SGD
with tf.device('/cpu:0'):
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    NUM_EPOCHS = 50000
    BATCH_SIZE = 1000
    for epoch in range(NUM_EPOCHS):
        # Train with each example
        tic = time.time()
        num_iters = len(train_X) // BATCH_SIZE
        for i in range(num_iters):
            idx1 = BATCH_SIZE * i
            idx2 = idx1 + BATCH_SIZE
            sess.run(updates, feed_dict={X: train_X[idx1:idx2], y: train_y[idx1:idx2]})
        
        h_out, pred_train = sess.run([h, yhat], feed_dict={X: train_X, y: train_y})
        train_error = 0 # TODO
        
        pred_test = sess.run(yhat, feed_dict={X: test_X, y: test_y})
        test_error = 0 # TODO
        toc = time.time()
        if epoch % 25 == 0:
            print('time:', toc-tic)
            print(np.mean(pred_train))
            print("Epoch = %d, train error = %.2f, test error = %.2f"
                  % (epoch + 1, train_error, test_error))
    sess.close()
